In [14]:
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import Document
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from typing import List
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [15]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = ChatOllama(model = "deepseek-r1:7b", callbacks=callback_manager)

In [9]:
plan_template = """ 
I need you to help me break down the following long-form writing instruction into multiple subtasks. Each subtask will guide the writing of one paragraph in the essay, and should include the main points and word count requirements for that paragraph.

The writing instruction is as follows:

{intructions}

Please break it down in the following format, with each subtask taking up one line:

Paragraph 1 - Main Point: [Describe the main point of the paragraph, in detail] - Word Count: [Word count requirement, e.g., 400 words]

Paragraph 2 - Main Point: [Describe the main point of the paragraph, in detail] - Word Count: [word count requirement, e.g. 1000 words].

...

Make sure that each subtask is clear and specific, and that all subtasks cover the entire content of the writing instruction. Do not split the subtasks too finely; each subtask's paragraph should be no less than 200 words and no more than 1000 words. Do not output any other content. As this is an ongoing work, omit open-ended conclusions or other rhetorical hooks.

"""

In [21]:
write_template = """ 
You are an excellent writing assistant. I will give you an original writing instruction and my planned writing steps. I will also provide you with the text I have already written. Please help me continue writing the next paragraph based on the writing instruction, writing steps, and the already written text.

Writing instruction:

{intructions}

Writing steps:

{plan}

Already written text:

{text}

Please integrate the original writing instruction, writing steps, and the already written text, and now continue writing {STEP}. If needed, you can add a small subtitle at the beginning. Remember to only output the paragraph you write, without repeating the already written text.
"""

In [17]:
plan_prompt = ChatPromptTemplate([
        ('user', plan_template)
    ])

In [18]:
plan_chain = plan_prompt | llm | StrOutputParser()

In [19]:
if __name__ == "__main__":
    test_instruction = "Give me an indepth plan on how to write tests with pytest. I want to learn how to do TDD."
    
    result = plan_chain.invoke({"intructions": test_instruction})
    
    # Print the result
    print("Generated Writing Plan:")
    print(result)

<think>
Okay, so I'm trying to understand how to break down the given writing instruction into multiple subtasks using the specified format. The user wants a detailed plan on how to write tests with pytest and learn TDD (Test-Driven Development). They provided an example response that splits this into seven paragraphs with specific main points and word counts.

First, I need to parse their original instruction carefully: "Give me an indepth plan on how to write tests with pytest. I want to learn how to do TDD." So the two main topics are writing unit tests using pytest and understanding TDD principles.

The example response splits this into seven subtasks:

1. Setting up the environment for testing
2. Writing first failing tests
3. Implementing test-driven development
4. Using pytest syntax and conventions
5. Managing dependencies with dependency injection
6. Best practices in writing unit tests
7. Continuous integration setup

Each of these subtasks is given a main point, word count, 

In [22]:
write_prompt = ChatPromptTemplate([
    ('user', write_template)
    ])

write_chain = write_prompt | llm | StrOutputParser()

In [50]:
if __name__ == "__main__":
    test_instruction = "Write a 1500-word essay about the impact of artificial intelligence on modern society, covering its benefits, potential risks, and ethical considerations."
    test_plan = "Paragraph 1 - Main Point: Introduction to AI and its growing influence - Word Count: 200 words"
    test_text = "Artificial Intelligence (AI) has become an integral part of our modern society, revolutionizing various aspects of our daily lives and industries."
    
    result = write_chain.invoke({
        "intructions": test_instruction,
        "plan": test_plan,
        "text": test_text,
        "STEP": "Paragraph 1"
    })
    
    print("Generated Paragraph:")
    print(result)

<think>
Okay, so I need to help this user expand their introduction on AI. They have some existing text that's about how AI is an integral part of modern society and has revolutionized daily lives and industries. The next step is to write the first paragraph, which should be around 200 words.

First, I'll start by understanding what they want. Their instruction is a 1500-word essay on AI's impact, covering benefits, risks, and ethics. They’re just starting with the introduction, so this paragraph needs to set the stage for the rest of the essay.

They already have some good points: revolutionizing daily lives and industries, being integral in various aspects like healthcare, transportation, etc., solving problems efficiently, but also raising concerns about job displacement and ethical issues.

I should integrate these ideas smoothly. Maybe begin with a general statement about AI's influence, then touch on its benefits in specific areas, mention the growth of AI as technology advances,

In [25]:
def write_markdown_file(content, filename):
    with open(f"{filename}.md", "w") as f:
        f.write(content)

In [53]:
def planning_node(state):
    """take the initial prompt and write a plan to make a long doc"""
    print("---PLANNING THE WRITING---")
    initial_prompt = state['initial_prompt']
    num_steps = int(state['num_steps'])
    num_steps += 1

    plan = plan_chain.invoke({"intructions": initial_prompt})

    return {"plan": plan, "num_steps":num_steps}

In [55]:
def count_words(text):
        """
        Count the number of words in the given text.
        
        Args:
            text (str): The input text to count words from.
        
        Returns:
            int: The number of words in the text.
        """
        # Split the text into words and count them
        words = text.split()
        return len(words)

def writing_node(state):
    """take the initial prompt and write a plan to make a long doc"""
    print("---WRITING THE DOC---")
    initial_instruction = state['initial_prompt']
    plan = state['plan']
    num_steps = int(state['num_steps'])
    num_steps += 1

    plan = plan.strip().replace('\n\n', '\n')
    planning_steps = plan.split('\n')
    text = ""
    responses = []
    if len(planning_steps) > 50:
        print("plan is too long")
        # print(plan)
        return
    for idx,step in enumerate(planning_steps):
        # Invoke the write_chain
        result = write_chain.invoke({
            "intructions": initial_instruction,
            "plan": plan,
            "text": text,
            "STEP": step
        })
        # result = step
        # print(f"----------------------------{idx}----------------------------")
        # print(step)
        # print("----------------------------\n\n")
        responses.append(result)
        text += result + '\n\n'

    final_doc = '\n\n'.join(responses)

    # Count words in the final document
    word_count = count_words(final_doc)
    print(f"Total word count: {word_count}")

    return {"final_doc": final_doc, "word_count": word_count, "num_steps":num_steps}

In [54]:
def saving_node(state):
    """take the finished long doc and save it to local disk as a .md file   """
    print("---SAVING THE DOC---")
    initial_prompt = state['initial_prompt']
    plan = state['plan']
    final_doc = state['final_doc']
    word_count = state['word_count']
    llm_name = state['llm_name']
    num_steps = int(state['num_steps'])
    num_steps += 1

    final_doc = final_doc + f"\n\nTotal word count: {word_count}"

    # save to local disk
    write_markdown_file(final_doc, f"final_doc")
    write_markdown_file(plan, f"plan")

    return { "num_steps":num_steps}

In [56]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_prompt: initial prompt
        plan: plan
        num_steps: number of steps
        llm_name: name of the LLM
        word_count: word count of the final document
    """
    initial_prompt : str
    plan : str
    num_steps : int
    final_doc : str
    write_steps : List[str]
    word_count : int

In [57]:
def create_workflow(llm):
    workflow = StateGraph(GraphState)
    workflow.add_node("planning_node", planning_node)
    workflow.add_node("writing_node", writing_node)
    workflow.add_node("saving_node", saving_node)
    workflow.set_entry_point("planning_node")
    workflow.add_edge("planning_node", "writing_node")
    workflow.add_edge("writing_node", "saving_node")
    workflow.add_edge("saving_node", END)
    return workflow.compile()

In [58]:
test_instruction = """
Please write a 5000 PRD on building a minimalistic AI powered note takking system that uses internet search
for context and tool to be used with an LLM. It also has a feature to quiz the user about the content
it generates so the user can remember the content better.
"""

inputs = {"initial_prompt": test_instruction,  
          "num_steps": 0,}

In [59]:
app = create_workflow(llm)

In [60]:
output = app.invoke(inputs)

---PLANNING THE WRITING---
<think>
Alright, so I need to break down this writing instruction into multiple subtasks for a PRD of 5000 words on building a minimalistic AI-powered note-taking system. The user also wants features that use the internet context with an LLM and include a quiz function to help remember content better.

First, let me understand what each part of the instruction entails. The main goal is to outline the PRD into clear subtasks, each focusing on one paragraph. Each should have a main point, detailed description, and a word count between 200-1000 words.

I'll start by identifying key components that need to be covered in the PRD. Probably, it's best to structure this logically:

1. **Introduction**: Overview of the system's purpose, features, and benefits.
2. **System Architecture**: The technical backbone including user interface, AI integration, data storage.
3. **Context Retrieval Mechanism**: How the system uses the internet for context.
4. **AI-Powered Note-T

KeyError: 'llm_name'